In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
from tqdm import tqdm

# =============================
# 1. ĐƯỜNG DẪN DỮ LIỆU
# =============================
train_dir = "/content/drive/My Drive/data_chia/train"
test_dir  = "/content/drive/My Drive/data_chia/test"

# =============================
# 2. TRANSFORM
# =============================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# =============================
# 3. LOAD DATA
# =============================
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Số lượng ảnh train:", len(train_dataset))
print("Số lượng ảnh test:", len(test_dataset))
print("Classes:", train_dataset.classes)

# =============================
# 4. RESNET18
# =============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# =============================
# 5. TRAIN + TEST LOOP
# =============================
def train_one_epoch(epoch):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch} Training")

    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loop.set_postfix(loss=running_loss/total, acc=100*correct/total)

def evaluate():
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Accuracy Test: {acc:.2f}%")
    return acc


# =============================
# 6. TRAINING
# =============================
epochs = 10
best_acc = 0

for epoch in range(1, epochs+1):
    train_one_epoch(epoch)
    acc = evaluate()

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "/content/drive/MyDrive/resnet18_parkinson_best.pth")
        print(">> Saved new best model!")

print("Training hoàn tất!")
print(f"Best Accuracy: {best_acc:.2f}%")


Số lượng ảnh train: 2610
Số lượng ảnh test: 654
Classes: ['Healthy', 'Parkinson']


Epoch 1 Training: 100%|██████████| 82/82 [05:36<00:00,  4.10s/it, acc=82, loss=0.0127]


Accuracy Test: 91.59%
>> Saved new best model!


Epoch 2 Training: 100%|██████████| 82/82 [00:26<00:00,  3.06it/s, acc=93, loss=0.0056]


Accuracy Test: 97.09%
>> Saved new best model!


...

Accuracy Test: 98.62%


Epoch 9 Training: 100%|██████████| 82/82 [00:26<00:00,  3.09it/s, acc=98.5, loss=0.00127]


Accuracy Test: 98.78%


Epoch 10 Training: 100%|██████████| 82/82 [00:26<00:00,  3.08it/s, acc=98.3, loss=0.00141]


Accuracy Test: 99.08%
Training hoàn tất!
Best Accuracy: 99.24%


In [9]:
!ls "/content/drive/My Drive/Colab Notebooks"


best_resnet18_focal.pth     data.zip
best_resnet18_ham10000.pth  skin_cancer_resnet18_finetune.pth
best_resnet50_focal.pth     skin_cancer_resnet18.pth
car_dataset.csv		    Untitled0.ipynb
data_final		    Untitled1.ipynb
data_mini.zip
